<a href="https://colab.research.google.com/github/prasad-dash/COVID-Model/blob/main/PuchoInternshipSubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Procuring the dataset

In [ ]:
!wget https://zenodo.org/record/4498364/files/public_dataset.zip?download=1
!unzip /content/public_dataset.zip?download=1
!cp ./public_dataset/metadata_compiled.csv /content
import warnings
warnings.filterwarnings('ignore')

##Importing Libraries


*   librosa
*   numpy
*   pandas
*   shutil
*   os
*   matplotlib.pyplot
*   sklearn
*   scipy




In [ ]:
import os
import numpy as np
import librosa as lb
import pandas as pd
import shutil
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

##Exploratory Data Analysis

In [ ]:
#The metadata_compiled file was moved to content directory for ease of working
!cp /content/public_dataset/metadata_compiled.csv
meta_dataset=pd.read_csv("/content/metadata_compiled.csv")
SIZE=len(meta_dataset)
for column in meta_dataset.columns:
  print(f'{column}:{meta_dataset[column].isna().sum()}')

cp: missing destination file operand after '/content/public_dataset/metadata_compiled.csv'
Try 'cp --help' for more information.
uuid:0
datetime:0
cough_detected:0
SNR:0
latitude:11466
longitude:11466
age:12332
gender:11326
respiratory_condition:11326
fever_muscle_pain:11326
status:11326
quality_1:26730
cough_type_1:26748
dyspnea_1:26730
wheezing_1:26730
stridor_1:26730
choking_1:26730
congestion_1:26730
nothing_1:26730
diagnosis_1:26748
severity_1:26748
quality_2:26730
cough_type_2:26749
dyspnea_2:26730
wheezing_2:26730
stridor_2:26730
choking_2:26730
congestion_2:26730
nothing_2:26730
diagnosis_2:26748
severity_2:26748
quality_3:26730
cough_type_3:26755
dyspnea_3:26730
wheezing_3:26730
stridor_3:26730
choking_3:26730
congestion_3:26730
nothing_3:26730
diagnosis_3:26758
severity_3:26755
quality_4:26730
cough_type_4:26749
dyspnea_4:26730
wheezing_4:26730
stridor_4:26730
choking_4:26730
congestion_4:26730
nothing_4:26730
diagnosis_4:26761
severity_4:26751


In [ ]:
SIZE=len(meta_dataset)

In [ ]:
meta_dataset.head(10)

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,quality_1,cough_type_1,dyspnea_1,wheezing_1,stridor_1,choking_1,congestion_1,nothing_1,diagnosis_1,severity_1,quality_2,cough_type_2,dyspnea_2,wheezing_2,stridor_2,choking_2,congestion_2,nothing_2,diagnosis_2,severity_2,quality_3,cough_type_3,dyspnea_3,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,diagnosis_3,severity_3,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,2020-11-25T18:58:50.488301+00:00,0.0155,7.326171,48.9,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,0.000000,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,NaN,NaN,21.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,23.014715,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,NaN,female,False,True,COVID-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,001e2f19-d81c-4029-b33c-d2db56b23a4a,2020-10-18T15:11:02.106636+00:00,0.7811,12.566406,45.7,4.9,20.0,male,True,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,00273cdf-ed90-4105-84ec-0c88d52f1dc0,2020-05-15T08:50:59.481638+00:00,0.0307,0.000000,43.6,-6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Using Cough sound to detect covid

In [ ]:
covDataset=meta_dataset.iloc[meta_dataset.status.dropna().index]

In [ ]:
covDataset.head()

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,quality_1,cough_type_1,dyspnea_1,wheezing_1,stridor_1,choking_1,congestion_1,nothing_1,diagnosis_1,severity_1,quality_2,cough_type_2,dyspnea_2,wheezing_2,stridor_2,choking_2,congestion_2,nothing_2,diagnosis_2,severity_2,quality_3,cough_type_3,dyspnea_3,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,diagnosis_3,severity_3,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,NaN,NaN,21.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,NaN,female,False,True,COVID-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
covDataset.status.value_counts()


healthy        12479
symptomatic     2590
COVID-19        1155
Name: status, dtype: int64

In [ ]:
i=1
for col in covDataset.columns:
  print(f'{i}.{col}: {covDataset[col].isna().sum()}')
  i+=1

1.uuid: 0
2.datetime: 0
3.cough_detected: 0
4.SNR: 0
5.latitude: 6487
6.longitude: 6487
7.age: 1006
8.gender: 0
9.respiratory_condition: 0
10.fever_muscle_pain: 0
11.status: 0
12.quality_1: 15527
13.cough_type_1: 15535
14.dyspnea_1: 15527
15.wheezing_1: 15527
16.stridor_1: 15527
17.choking_1: 15527
18.congestion_1: 15527
19.nothing_1: 15527
20.diagnosis_1: 15535
21.severity_1: 15535
22.quality_2: 15532
23.cough_type_2: 15543
24.dyspnea_2: 15532
25.wheezing_2: 15532
26.stridor_2: 15532
27.choking_2: 15532
28.congestion_2: 15532
29.nothing_2: 15532
30.diagnosis_2: 15542
31.severity_2: 15542
32.quality_3: 15536
33.cough_type_3: 15550
34.dyspnea_3: 15536
35.wheezing_3: 15536
36.stridor_3: 15536
37.choking_3: 15536
38.congestion_3: 15536
39.nothing_3: 15536
40.diagnosis_3: 15553
41.severity_3: 15550
42.quality_4: 15423
43.cough_type_4: 15442
44.dyspnea_4: 15423
45.wheezing_4: 15423
46.stridor_4: 15423
47.choking_4: 15423
48.congestion_4: 15423
49.nothing_4: 15423
50.diagnosis_4: 15454
51.se

###Preparing the training and test datasets


In [ ]:
files=[file for file in os.listdir("./public_dataset")]
covDataset=covDataset.iloc[:,[0,10]]

In [ ]:
covDataset.head()

,uuid,status
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,healthy
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,healthy
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,healthy
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,healthy
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,COVID-19


###Methods used:

The paper used in the suggested dataset involved thr use of MFCC along with other spectral feautures.I have extracted the MFCCs of the given samples after segmenting them into blobs of 8 seconds. I have built a simple deep-learning neural-net with four layers each with a relu activation function (the given code is written in sequential model but can also be presented in the functional model). There are total of 170343 parameters. Using only given data I am able to achive 75% validation accuracy is classifying the data as healthy ,symptomatic ,COVID-19 . I have also used k-fold cross vaidation while training. 

### Further possible impovements:

The used data has not been cleaned completely ie some samples which dont have any sound have been marked with one of the categories. Also the paper which puts together data which references another paper whcich uses a more complicared neural net with convolutions and residual layers to train the dataset along with more spectral feautures and other sound data. These improvements have not been implemented but can possibly increase the accuracy even more.

In [ ]:
X=[]
y=[]
segment_size=8
for i in range(int(len(covDataset)*2/3)):
  print(i)
  if (covDataset.iloc[i,0]+'.webm') in files:
    arr,sr=lb.load(f'./public_dataset/{covDataset.iloc[i,0]+".webm"}',sr=48000)
  if (covDataset.iloc[i,0]+'.ogg') in files:
    arr,sr=lb.load(f'./public_dataset/{covDataset.iloc[i,0]+".ogg"}',sr=48000)
  duration=lb.get_duration(arr,sr=48000)
  itime=0
  ftime=0
  for j in range(int(duration/segment_size)):
    itime=j*segment_size
    ftime=itime+segment_size
    feautures=lb.feature.mfcc(y=arr[itime:ftime], sr=sr, n_mfcc=10)
    X.append(feautures)
    y.append(covDataset.iloc[i,1])

In [ ]:
X=np.array(X)
X=X.reshape(X.shape[0],X.shape[1])
Le=preprocessing.LabelEncoder()
y_t=Le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_t, test_size=0.33, random_state=42)

In [ ]:
model=tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.InputLayer(10,))
model.add(tf.keras.layers.Dense(512,))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(3))
model.add(tf.keras.layers.Softmax())


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 512)               5632      
_________________________________________________________________
re_lu_48 (ReLU)              (None, 512)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 256)               131328    
_________________________________________________________________
re_lu_49 (ReLU)              (None, 256)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 128)               32896     
_________________________________________________________________
re_lu_50 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 3)               

In [ ]:
model.fit(x=X_train,y=y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
153/153 [==============================] - 1s 6ms/step - loss: 8.3078 - accuracy: 0.6233 - val_loss: 4.8313 - val_accuracy: 0.7561
Epoch 2/100
153/153 [==============================] - 1s 5ms/step - loss: 1.5074 - accuracy: 0.6789 - val_loss: 0.8397 - val_accuracy: 0.7561
Epoch 3/100
153/153 [==============================] - 1s 6ms/step - loss: 0.9203 - accuracy: 0.7208 - val_loss: 0.7667 - val_accuracy: 0.7561
Epoch 4/100
153/153 [==============================] - 1s 5ms/step - loss: 0.9998 - accuracy: 0.7159 - val_loss: 0.7144 - val_accuracy: 0.7561
Epoch 5/100
153/153 [==============================] - 1s 6ms/step - loss: 0.8381 - accuracy: 0.7276 - val_loss: 0.7465 - val_accuracy: 0.7561
Epoch 6/100
153/153 [==============================] - 1s 5ms/step - loss: 0.8454 - accuracy: 0.7349 - val_loss: 0.7294 - val_accuracy: 0.7561
Epoch 7/100
153/153 [==============================] - 1s 5ms/step - loss: 0.7445 - accuracy: 0.7631 - val_loss: 0.7430 - val_accuracy: 0.7561

###K-fold cross validation

In [ ]:
len(y_t)

986

In [ ]:
Kmodel=tf.keras.models.Sequential()
Kmodel.add(tf.keras.layers.InputLayer(10,))
Kmodel.add(tf.keras.layers.Dense(512,))
Kmodel.add(tf.keras.layers.ReLU())
Kmodel.add(tf.keras.layers.Dense(256))
Kmodel.add(tf.keras.layers.ReLU())
Kmodel.add(tf.keras.layers.Dense(128))
Kmodel.add(tf.keras.layers.ReLU())
Kmodel.add(tf.keras.layers.Dense(3))
Kmodel.add(tf.keras.layers.Softmax())
Kmodel.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True)
fold_no=0
for train, test in kfold.split(X, y_t):
  Kmodel.fit(X[train], y_t[train],batch_size=32,epochs=100,validation_data=(X[test],y_t[test]))


Epoch 1/100
206/206 [==============================] - 1s 5ms/step - loss: 4.8078 - accuracy: 0.6114 - val_loss: 0.8342 - val_accuracy: 0.7743
Epoch 2/100
206/206 [==============================] - 1s 5ms/step - loss: 1.2056 - accuracy: 0.6728 - val_loss: 0.7481 - val_accuracy: 0.7743
Epoch 3/100
206/206 [==============================] - 1s 5ms/step - loss: 0.9687 - accuracy: 0.7123 - val_loss: 1.4647 - val_accuracy: 0.1532
Epoch 4/100
206/206 [==============================] - 1s 5ms/step - loss: 0.9300 - accuracy: 0.7036 - val_loss: 0.8134 - val_accuracy: 0.7743
Epoch 5/100
206/206 [==============================] - 1s 5ms/step - loss: 0.7935 - accuracy: 0.7414 - val_loss: 0.7082 - val_accuracy: 0.7743
Epoch 6/100
206/206 [==============================] - 1s 5ms/step - loss: 0.7640 - accuracy: 0.7549 - val_loss: 0.7733 - val_accuracy: 0.7743
Epoch 7/100
206/206 [==============================] - 1s 4ms/step - loss: 0.7403 - accuracy: 0.7621 - val_loss: 0.6918 - val_accuracy: 0.7743

Clearly K-fold cross validation impoved the results by 3% which can be imporoved even more by data cleaning



In [ ]:
X_test=[]
y_test=[]
segment_size=8
for i in range(int(len(covDataset)*2/3),int(len(covDataset))):
  print(i)
  if (covDataset.iloc[i,0]+'.webm') in files:
    arr,sr=lb.load(f'./public_dataset/{covDataset.iloc[i,0]+".webm"}',sr=48000)
  if (covDataset.iloc[i,0]+'.ogg') in files:
    arr,sr=lb.load(f'./public_dataset/{covDataset.iloc[i,0]+".ogg"}',sr=48000)
  duration=lb.get_duration(arr,sr=48000)
  itime=0
  ftime=0
  for j in range(int(duration/segment_size)):
    itime=j*segment_size
    ftime=itime+segment_size
    feautures=lb.feature.mfcc(y=arr[itime:ftime], sr=sr, n_mfcc=10)
    X_test.append(feautures)
    y_test.append(covDataset.iloc[i,1])

In [ ]:
X_test=np.array(X_test)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
Le_test=preprocessing.LabelEncoder()
ytest=Le_test.fit_transform(y_test)

In [ ]:
preds=Kmodel.predict(X_test)

In [ ]:
preds.shape

(3708, 3)

In [ ]:
preds=[pred.argmax() for pred in preds]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(ytest, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       273
           1       0.77      1.00      0.87      2854
           2       0.00      0.00      0.00       581

    accuracy                           0.77      3708
   macro avg       0.26      0.33      0.29      3708
weighted avg       0.59      0.77      0.67      3708



###From the classification score it is clear that the model is able to find the model can find healthy people more easily and is also presenting some Covid19 results as positive also this may be attributed to the fact that the given dataset has many more healthy results as compared to covid19 or symptomatic almost 10:1 which is not healthy.

In [ ]:
Le_test.classes_

array(['COVID-19', 'healthy', 'symptomatic'], dtype='<U11')